# Data Pre-processing

In [1]:
# import torch
# if torch.cuda.is_available():
#     device = torch.device('cuda:1')
# torch.cuda.set_device(1)
# torch.cuda.current_device()

1

In [2]:
# Read the data:
import json

with open('data/eng_math_train.json') as f:
    math_train = json.load(f)
with open('data/eng_math_test.json') as f:
    math_test = json.load(f)
    

In [3]:
# Data processing, add answer D to each question
def Add_answer(data):
    for i in range(len(data)):
        exercise = data[i]
        if len(exercise['choices']) < 4:
            exercise['choices'].append('D. None of the above')
Add_answer(math_train['data'])
Add_answer(math_test['data'])


In [4]:
from datasets import Dataset
# Change from dict to dataset
train_ds = Dataset.from_list(math_train['data'][:960])
eval_ds = Dataset.from_list(math_train['data'][960:])
test_ds = Dataset.from_list(math_test['data'])

# Add the model

In [5]:
model_dir = "roberta-base"
from transformers import AutoTokenizer, RobertaForMultipleChoice

tokenizer = AutoTokenizer.from_pretrained("roberta-base")
model = RobertaForMultipleChoice.from_pretrained("roberta-base")

Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.weight', 'classifier.bias', 'roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
# from transformers import AutoTokenizer

# model_dir = "microsoft/deberta-v2-xlarge"
# tokenizer = AutoTokenizer.from_pretrained(model_dir)

# from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
# model = AutoModelForMultipleChoice.from_pretrained(model_dir).to(device)

In [7]:
# from transformers import AutoModelForCausalLM, AutoTokenizer, AutoModelForMultipleChoice
# from transformers.generation import GenerationConfig


# tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-1_8B", trust_remote_code=True)

# model = AutoModelForMultipleChoice.from_pretrained("Qwen/Qwen-1_8B", device_map="auto", trust_remote_code=True).to(device)

In [8]:
# We'll create a dictionary to convert option names (A, B, C, D) into indices and back again
options = 'ABCD'
indices = list(range(4))

option_to_index = {option: index for option, index in zip(options, indices)}
index_to_option = {index: option for option, index in zip(options, indices)}

def preprocess(example):
    first_sentence = [example['question']] * 4
    second_sentence = []
    for choice in example['choices']:
        second_sentence.append(choice)
    # Our tokenizer will turn our text into token IDs model can understant
    tokenized_example = tokenizer(first_sentence, second_sentence, truncation=True)
    tokenized_example['label'] = option_to_index[example['answer'][0]] #Label the id according to option
    return tokenized_example

tokenized_train_ds = train_ds.map(preprocess, batched=False, remove_columns=['question', 'choices', 'explanation', 'answer'])
tokenized_eval_ds = eval_ds.map(preprocess, batched=False, remove_columns=['question', 'choices', 'explanation', 'answer'])

Map:   0%|          | 0/960 [00:00<?, ? examples/s]

Map:   0%|          | 0/240 [00:00<?, ? examples/s]

In [9]:
# Following datacollator from https://huggingface.co/docs/transformers/tasks/multiple_choice
from dataclasses import dataclass
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from typing import Optional, Union
import torch

@dataclass
class DataCollatorForMultipleChoice:
    tokenizer: PreTrainedTokenizerBase
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    
    def __call__(self, features):
        label_name = "label" if 'label' in features[0].keys() else 'labels'
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]['input_ids'])
        flattened_features = [
            [{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features
        ]
        flattened_features = sum(flattened_features, [])
        
        batch = self.tokenizer.pad(
            flattened_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors='pt',
        )
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        batch['labels'] = torch.tensor(labels, dtype=torch.int64)
        return batch


In [10]:
from transformers import TrainingArguments, Trainer

model_dir = 'finetuned'
training_args = TrainingArguments(
    output_dir=model_dir,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=10,
    weight_decay=0.01,
    report_to='none'
)

In [11]:
training_args.device

device(type='cuda', index=2)

In [12]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_eval_ds,
    tokenizer=tokenizer,
    data_collator=DataCollatorForMultipleChoice(tokenizer=tokenizer),
)

In [13]:
trainer.train()

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss
1,No log,1.386226
2,No log,1.386295
3,No log,1.386293
4,No log,1.386295
5,No log,1.386294
6,No log,1.386294
7,1.390900,1.386294
8,1.390900,1.386295
9,1.390900,1.386294
10,1.390900,1.386294


TrainOutput(global_step=800, training_loss=1.3903372192382812, metrics={'train_runtime': 241.9349, 'train_samples_per_second': 39.68, 'train_steps_per_second': 3.307, 'total_flos': 1089689318466336.0, 'train_loss': 1.3903372192382812, 'epoch': 10.0})

In [22]:
trainer.save_model("model/Theorem_mind_roberta")